In [2]:
from schnapsen.game import SchnapsenGamePlayEngine, Bot, Rank
import schnapsen
from schnapsen.bots.rdeep import RdeepBot
from Rdeephighpriority import RdeephighpriorityBot
import random



def play_games(num_games: int, bot1: Bot, bot2: Bot) -> tuple[int, int]:
    engine = SchnapsenGamePlayEngine()
    bot1_wins = 0
    bot2_wins = 0

    for i in range(num_games):
        if i % 2 == 0:
            # Play game with bot1 as the leader
            result = engine.play_game(bot1, bot2, rng=random.Random(i))
        else:
            # Play game with bot2 as the leader
            result = engine.play_game(bot2, bot1, rng=random.Random(i))

        # Extract the winner
        winner = result[0]  # Use indexing or unpacking based on the return structure
        if winner is bot1:
            bot1_wins += 1
        else:
            bot2_wins += 1

        print(f"Game {i + 1}/{num_games}: Winner - {'Bot1' if winner is bot1 else 'Bot2'}")

    return bot1_wins, bot2_wins


# Set up the bots
random_seed = 42
num_samples = 100
depth = 5

rand = random.Random(random_seed)
bot1 = RdeepBot(num_samples=num_samples, depth=depth, rand=rand, name="RdeepBot")
bot2 = RdeephighpriorityBot(num_samples=num_samples, depth=depth, rand=rand, name="RdeephighpriorityBot")

# Play games
num_games = 100
bot1_wins, bot2_wins = play_games(num_games, bot1, bot2)

# Display results
print("\nRESULTS:")
print(f"RdeepBot wins: {bot1_wins}")
print(f"RdeephighpriorityBot wins: {bot2_wins}")
print(f"Winning percentage for RdeepBot: {bot1_wins / num_games * 100:.2f}%")
print(f"Winning percentage for RdeephighpriorityBot: {bot2_wins / num_games * 100:.2f}%")


ModuleNotFoundError: No module named 'schnapsen'

In [6]:
import random
from schnapsen.game import SchnapsenGamePlayEngine, Bot
from schnapsen.bots.Minimaxhighpriority import MinimaxhighpriorityBot
from schnapsen.bots.minimax import MiniMaxBot
from schnapsen.bots.TwoPhaseBot import TwoPhaseBot


def play_games(num_games: int, bot1: Bot, bot2: Bot) -> tuple[int, int]:
    """
    Plays a series of games between two bots and returns the number of wins for each bot.

    :param num_games: The number of games to play
    :param bot1: The first bot
    :param bot2: The second bot
    :return: A tuple containing the number of wins for bot1 and bot2
    """
    engine = SchnapsenGamePlayEngine()
    bot1_wins = 0
    bot2_wins = 0

    for i in range(num_games):
        if i % 2 == 0:
            # Bot1 starts as the leader in even games
            result = engine.play_game(bot1, bot2, rng=random.Random(i))
        else:
            # Bot2 starts as the leader in odd games
            result = engine.play_game(bot2, bot1, rng=random.Random(i))

        # Extract the winner
        winner = result[0]  # Get the winner
        if winner is bot1:
            bot1_wins += 1
        else:
            bot2_wins += 1

        print(f"Game {i + 1}/{num_games}: Winner - {'Bot1' if winner is bot1 else 'Bot2'}")

    return bot1_wins, bot2_wins



# Set up the random seed
random_seed = 42
rand = random.Random(random_seed)

# Wrap MiniMaxBot and MinimaxhighpriorityBot with TwoPhaseBot
bot1 = TwoPhaseBot(phase_two_bot=MinimaxhighpriorityBot(name="MinimaxhighpriorityBot"), rand=rand, name="TwoPhaseBot1")
bot2 = TwoPhaseBot(phase_two_bot=MiniMaxBot(name="MiniMaxBot"), rand=rand, name="TwoPhaseBot2")

# Play games
num_games = 100
bot1_wins, bot2_wins = play_games(num_games, bot1, bot2)

# Display results
print("\nRESULTS:")
print(f"MinimaxhighpriorityBot wins: {bot1_wins}")
print(f"MiniMaxBot wins: {bot2_wins}")
print(f"Winning percentage for MinimaxhighpriorityBot: {bot1_wins / num_games * 100:.2f}%")
print(f"Winning percentage for MiniMaxBot: {bot2_wins / num_games * 100:.2f}%")



ModuleNotFoundError: No module named 'schnapsen'

In [3]:
import random
import pathlib
from typing import Optional

import click
from schnapsen.bots import RandBot, RdeepBot
from schnapsen.game import Bot, GamePlayEngine, SchnapsenGamePlayEngine,

@click.group()
def main() -> None:
    """Various Schnapsen Game Examples"""


def play_games_and_return_stats(engine: GamePlayEngine, bot1: Bot, bot2: Bot, pairs_of_games: int) -> int:
    """
    Play 2 * pairs_of_games games between bot1 and bot2, and return how often bot1 won.
    Each pair of games swaps the roles of leader and follower.
    """
    bot1_wins: int = 0
    for game_pair in range(pairs_of_games):
        for lead, follower in [(bot1, bot2), (bot2, bot1)]:
            winner, _, _ = engine.play_game(lead, follower, random.Random(game_pair))
            if winner == bot1:
                bot1_wins += 1
        if game_pair > 0 and (game_pair + 1) % 10 == 0:
            print(f"Progress: {game_pair + 1}/{pairs_of_games} game pairs played")
    return bot1_wins


# High-value prioritization logic for bots
class HighValueRandBot(RandBot):
    def get_move(self, perspective, leader_move: Optional[Move]) -> Move:
        moves = perspective.valid_moves()
        high_value_moves = [move for move in moves if move.card.rank in ['A', '10']]
        if high_value_moves:
            return self.rng.choice(high_value_moves)
        return super().get_move(perspective, leader_move)


class HighValueRdeepBot(RdeepBot):
    def __evaluate(self, gamestate, engine, leader_move, my_move):
        high_value_bonus = sum(1 for card in gamestate.get_cards() if card.rank in ['A', '10'])
        base_score = super().__evaluate(gamestate, engine, leader_move, my_move)
        return base_score + high_value_bonus


class HighValueMinimaxBot(Bot):
    # A simple implementation of Minimax with high-value prioritization
    def get_move(self, perspective, leader_move: Optional[Move]) -> Move:
        valid_moves = perspective.valid_moves()
        high_value_moves = [move for move in valid_moves if move.card.rank in ['A', '10']]
        return max(high_value_moves or valid_moves, key=lambda move: move.card.points)


@main.command()
def compare_bots() -> None:
    engine = SchnapsenGamePlayEngine()

    # Bots to compare
    rand_bot = RandBot(random.Random(12345))
    high_value_rand_bot = HighValueRandBot(random.Random(54321))

    rdeep_bot = RdeepBot(num_samples=16, depth=4, rand=random.Random(12345))
    high_value_rdeep_bot = HighValueRdeepBot(num_samples=16, depth=4, rand=random.Random(54321))

    minimax_bot = Bot()  # Replace with actual MinimaxBot if implemented
    high_value_minimax_bot = HighValueMinimaxBot()

    # Number of game pairs
    pairs_of_games = 10

    # Run comparisons
    print("Comparing RandBot vs HighValueRandBot")
    randbot_wins = play_games_and_return_stats(engine, rand_bot, high_value_rand_bot, pairs_of_games)
    print(f"RandBot won {randbot_wins} out of {2 * pairs_of_games} games.")

    print("Comparing RdeepBot vs HighValueRdeepBot")
    rdeepbot_wins = play_games_and_return_stats(engine, rdeep_bot, high_value_rdeep_bot, pairs_of_games)
    print(f"RdeepBot won {rdeepbot_wins} out of {2 * pairs_of_games} games.")

    print("Comparing MinimaxBot vs HighValueMinimaxBot")
    minimax_wins = play_games_and_return_stats(engine, minimax_bot, high_value_minimax_bot, pairs_of_games)
    print(f"MinimaxBot won {minimax_wins} out of {2 * pairs_of_games} games.")


if _name_ == "_main_":
    main()

ModuleNotFoundError: No module named 'schnapsen'

In [ ]:

        # Get the list of valid moves and sort them by rank order
        moves = perspective.valid_moves()
        moves.sort(key=lambda move: rank_order[str(move.card.rank)], reverse=True)